In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from nflows.transforms.dequantization import UniformDequantization
from nflows.transforms.dequantization import VariationalDequantization

In [4]:
from nflows.distributions.normal import ConditionalDiagonalNormal
from nflows.distributions.normal import StandardNormal
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform

In [5]:
# Generate some dist with 2 real and 2 discrete dims

# These are the probabilities to draw a particular label
x_prob_1 = np.random.rand(10)
x_prob_2 = np.random.rand(5)
p_labels_1 = x_prob_1/(np.sum(x_prob_1))
p_labels_2 = x_prob_2/(np.sum(x_prob_2))
p_cum_labels_1 = np.cumsum(p_labels_1)
p_cum_labels_2 = np.cumsum(p_labels_2)

def p(x,y):
    if np.all(x > 0) and np.all(x < 1) and x.shape == y.shape:
        return m.exp(-(x[0] + x[1])/2) * np.cos((x[0]*y[0] + x[1]*y[1])*m.pi)
    return 0

def generate(n):
    x = np.zeros((n,4))

    counter = 0
    while(counter < n):
        y = np.zeros(2)
        y[0] = np.argmax(r.random() < p_cum_labels_1)
        y[1] = np.argmax(r.random() < p_cum_labels_2)

        while(True):
            # Get a point
            x_trial = np.random.rand(2)
            if r.random() < p(x_trial, y):
                x[counter][0] = x_trial[0]
                x[counter][1] = x_trial[1]
                x[counter][2] = y[0]
                x[counter][3] = y[1]
                counter += 1
                break

    return x

In [15]:
num_iter = 25000
batch_size = 500
for i in range(num_iter):
    x = generate(batch_size)
    x = torch.tensor(x, dtype=torch.float32)
    optimizer.zero_grad()
    loss = -flow.log_prob(inputs=x).mean()
    loss.backward()
    optimizer.step()
    
    if (i % 500 == 0):
        print(loss.item())

RuntimeError: output with shape [500] doesn't match the broadcast shape [500, 500]

In [8]:
vardeq = VariationalDequantization(max_labels=torch.tensor([-1,-1,9,4]), rqs_hidden_features=15, rqs_flow_layers=5)
unideq = UniformDequantization(max_labels=torch.tensor([-1,-1,9,4]))

In [13]:
'''
x_plot = generate(1000000)
x_select_1 = x_plot[x_plot[:,2] == 7]
x_select_2 = x_select_1[x_select_1[:,3] == 3]
plt.hist2d(x_select_2[:,0], x_select_2[:,1], bins=25)
plt.show()
'''

'\nx_plot = generate(1000000)\nx_select_1 = x_plot[x_plot[:,2] == 7]\nx_select_2 = x_select_1[x_select_1[:,3] == 3]\nplt.hist2d(x_select_2[:,0], x_select_2[:,1], bins=25)\nplt.show()\n'

In [14]:
'''
num_layers = 5
base_dist = BoxUniform(torch.zeros(3), torch.ones(3))

transforms = []
transforms.append(VariationalDequantization(max_labels=torch.tensor([-1,-1,9,4]), rqs_hidden_features=15, rqs_flow_layers=5))
for _ in range(num_layers):
    transforms.append(ReversePermutation(features=3))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=3, 
        hidden_features=25,
        num_bins=25,
        num_blocks=4,
    ))

transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist)
optimizer = optim.Adam(flow.parameters())
'''

'\nnum_layers = 5\nbase_dist = BoxUniform(torch.zeros(3), torch.ones(3))\n\ntransforms = []\ntransforms.append(VariationalDequantization(max_labels=torch.tensor([-1,-1,9,4]), rqs_hidden_features=15, rqs_flow_layers=5))\nfor _ in range(num_layers):\n    transforms.append(ReversePermutation(features=3))\n    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(\n        features=3, \n        hidden_features=25,\n        num_bins=25,\n        num_blocks=4,\n    ))\n\ntransform = CompositeTransform(transforms)\n\nflow = Flow(transform, base_dist)\noptimizer = optim.Adam(flow.parameters())\n'

tensor([[8.3695e-01, 3.3029e-01, 6.0000e+00, 3.0000e+00],
        [6.2797e-01, 1.5166e-01, 9.0000e+00, 1.0000e+00],
        [8.9222e-01, 2.1317e-01, 8.0000e+00, 4.0000e+00],
        [8.5068e-01, 5.0880e-01, 1.0000e+00, 2.0000e+00],
        [2.8508e-01, 5.2824e-01, 2.0000e+00, 3.0000e+00],
        [8.0762e-01, 2.1293e-01, 2.0000e+00, 1.0000e+00],
        [8.4560e-01, 4.7608e-01, 9.0000e+00, 1.0000e+00],
        [7.8080e-01, 5.3374e-01, 6.0000e+00, 3.0000e+00],
        [4.2927e-04, 9.0411e-01, 9.0000e+00, 0.0000e+00],
        [3.8161e-01, 1.1020e-01, 9.0000e+00, 3.0000e+00]])